In [1]:
from functions import *
from torch.optim import Adam

In [2]:
max_ = -1
for k,v in mapping.items():
    if v>max_:
        k_max = k
        max_ = v

In [4]:
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_lookup,len(k_max))

2.1029411764705874

In [4]:
max_

2.1029411764705874

In [2]:
from functions_pt import *

In [3]:
embed_size = 128
dropout = 0.1
multiple_factor = 1
layers = 2
batch_size = 64
agg = 'mean' # mean,sum,min,max
model = pointNet(layers,embed_size,agg,dropout,multiple_factor)#.to('cuda')
model.load_state_dict(torch.load('baseline_noBN.pt'))
model.eval()
model = model.to('cuda')

In [4]:
words_embed = np.load('possible_words_embed.npy')

In [8]:
# slightly worse than # of steps required by mapping as expected
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_model,len(k_max),model=model,words_embed=words_embed,top=0.6)

2.1249999999999987

In [2]:
# entropy policy
for i in np.linspace(0,7,15):
    steps = evaluate(matrix,np.arange(2309),policy_entropy,2309,factor=i)
    print('factor:{}, steps:{}'.format(i,steps))

factor:0.0, steps:3.5275010827197937
factor:0.5, steps:3.4638371589432673
factor:1.0, steps:3.4634040710264204
factor:1.5, steps:3.4634040710264204
factor:2.0, steps:3.4634040710264204
factor:2.5, steps:3.462970983109573
factor:3.0, steps:3.4638371589432673
factor:3.5, steps:3.4642702468601136


KeyboardInterrupt: 

In [2]:
# entropy policy with depth info
for i in np.linspace(1,2,5):
    steps = evaluate_depth(matrix,np.arange(2309),policy_entropy_depth,2309,factor=i,depth=1)
    print('factor:{}, steps:{}'.format(i,steps))

factor:1.0, steps:3.4634040710264204
factor:1.25, steps:3.462970983109573
factor:1.5, steps:3.4638371589432673
factor:1.75, steps:3.4642702468601136
factor:2.0, steps:3.4655695106106554


In [5]:
# see how well model does on full problem that it is not trained on
#     if count == 1: return 1
#     if count == 2: return 1.5
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.5266349068860987

RL Loop

In [6]:
lr = 6e-5
clip = 1
epochs = 2
damp = 1
iterations = 15
max_data = 2000

for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model,2309,0,model=model,words_embed=words_embed,top=0.6,
                         eps=0.2/(i+1)**2)
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    model.train()
    opt = Adam([    {'params': [p for p in model.parameters() if p is not model.w]},\
                {'params': model.w, 'lr': lr/4}
            ],lr=lr)
    for _ in range(epochs):
        yhat = model((words,length))
        loss = torch.mean(prob*(ys-yhat)**2)
        loss.backward()
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.664356864443484
iter:1, score:3.5478562148116075
iter:2, score:3.5279341706366414
iter:3, score:3.5071459506279794
iter:4, score:3.516673884798616
iter:5, score:3.515807708964922
iter:6, score:3.514075357297534
iter:7, score:3.5153746210480747
iter:8, score:3.518839324382852


KeyboardInterrupt: 

In [5]:
lr = 6e-5
clip = 1
epochs = 2
damp = 1
iterations = 15
max_data = 2000
states = set()
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model,2309,0,model=model,words_embed=words_embed,top=0.6,
                         eps=0.2/(i+1)**2)
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    model.train()
    opt = Adam([    {'params': [p for p in model.parameters() if p is not model.w]},\
                {'params': model.w, 'lr': lr/4}
            ],lr=lr)
    for j_ in range(epochs):
        yhat = model((words,length))
        loss = torch.mean(prob*(ys-yhat)**2)
        loss.backward()
        print('loss')
        print(j_,loss.cpu().item())
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.71892594196622
set size:398
loss
0 0.12590913474559784
loss
1 0.10894769430160522
iter:1, score:3.542226071892596
set size:715
loss
0 0.14993155002593994
loss
1 0.1562322974205017
iter:2, score:3.5179731485491574
set size:881
loss
0 0.177512064576149
loss
1 0.17453089356422424
iter:3, score:3.518406236466005
set size:994
loss
0 0.1868080198764801
loss
1 0.1805325299501419
iter:4, score:3.514075357297534
set size:1049
loss
0 0.1912974864244461
loss
1 0.19184242188930511
iter:5, score:3.515807708964922
set size:1052
loss
0 0.1924106776714325
loss
1 0.19461648166179657
iter:6, score:3.513209181463839
set size:1052
loss
0 0.19937260448932648
loss
1 0.19698752462863922


KeyboardInterrupt: 

In [5]:
lr = 7e-5
clip = 1
epochs = 2
damp = 1
iterations = 15
max_data = 0
states = set()
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model,2309,0,model=model,words_embed=words_embed,top=0.6,
                         eps=0.2/(i+1))
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    model.train()
    opt = Adam([    {'params': [p for p in model.parameters() if p is not model.w]},\
                {'params': model.w, 'lr': lr/4}
            ],lr=lr)
    for j_ in range(epochs):
        yhat = model((words,length))
        loss = torch.mean(prob*(ys-yhat)**2)
        loss.backward()
        print('loss')
        print(j_,loss.cpu().item())
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.6860112602858384
set size:390
loss
0 0.1273408979177475
iter:1, score:3.5560848852317037
set size:728
loss
0 0.20019201934337616
iter:2, score:3.522737115634477
set size:888
loss
0 0.2173539102077484
iter:3, score:3.5223040277176283
set size:1015
loss
0 0.2158091962337494
iter:4, score:3.515374621048075
set size:1059
loss
0 0.2277476042509079
iter:5, score:3.5162407968817693
set size:1079
loss
0 0.21915704011917114
iter:6, score:3.512776093546992
set size:1096
loss
0 0.2254505753517151
iter:7, score:3.514941533131228
set size:1109
loss
0 0.22602730989456177
iter:8, score:3.5140753572975334
set size:1122
loss
0 0.2215365171432495
iter:9, score:3.5179731485491574
set size:1122
loss
0 0.2132256031036377
iter:10, score:3.514941533131228
set size:1122
loss
0 0.22427460551261902
iter:11, score:3.5166738847986165
set size:1122
loss
0 0.21680185198783875
iter:12, score:3.512776093546992
set size:1123
loss
0 0.2189253866672516
iter:13, score:3.5166738847986165
set size:1123
loss